In [1]:
import os
import random as _random
import numpy

In [2]:
RND = 1
N_VAL = 5000
N_TRAIN = 500000
OUT_DIR = 'out_500k'

In [3]:
_random.seed(RND)
numpy.random.seed(RND)

In [4]:
%run '../data-generator.ipynb'

In [5]:
dg = DataGenerator('/d2/caches/tf-speech/train/audio')

# adjust mixing options
for k, v  in dg.mix_with.items():
    if '_background_noise_' in k:
        v['probability'] = 0.1

In [6]:
# get some stats on input data
counts = {k: len(v) for k, v in dg.input_files.items()}
print(counts)
del counts['unknown']
print('total samples for known words:', sum(counts.values()))
print('avg per known label:', sum(counts.values()) / len(counts))

{'yes': 2377, 'right': 2367, 'unknown': 41039, 'left': 2353, 'no': 2375, 'up': 2375, 'off': 2357, 'go': 2372, 'stop': 2380, 'on': 2367, 'down': 2359}
total samples for known words: 23682
avg per known label: 2368.2


In [7]:
if not os.path.isdir(OUT_DIR): os.makedirs(OUT_DIR)

In [8]:
def _gen():
    # generate validation set
    dg.val_files = {}
    val_X, val_Y = dg.generate_val_set(n=N_VAL)
    np.save('%s/val_files.npy' % OUT_DIR, dg.val_files)
    np.save('%s/val_X.npy' % OUT_DIR, val_X)
    np.save('%s/val_Y.npy' % OUT_DIR, val_Y)

    dg.generate_train_set(
        n_total=N_TRAIN,
        n_per_job=500,
        n_pools=16,
        X_file='%s/train_X.mem' % OUT_DIR,
        Y_file='%s/train_Y.mem' % OUT_DIR,
        tmp_dir='%s/train_tmp' % OUT_DIR)

In [9]:
%time _gen()

CPU times: user 5min 16s, sys: 15min 32s, total: 20min 48s
Wall time: 3h 31min 35s
